<a href="https://colab.research.google.com/github/PTson2207/Deployment-Model-App/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training someone model, then deploy ...

- Want: Save_model and then upload Google Storage
- Data: From Kaggle-Food 101
- Model(s): 10, 11, 12 classes

In [1]:
! nvidia-smi

Sat Mar 20 14:17:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Setup some Function

In [2]:
# thư viện
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
# Hàm unzip dowload_data file

def unzip_data(file_name):
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [4]:
# Setup dataset input
IMG_SIZE = (224, 224)

def create_data_loader(train_dir, test_dir, img_size=IMG_SIZE):
    train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode="categorical",
                                                                     image_size=img_size)
    
    test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                    label_mode="categorical",
                                                                    image_size=img_size)
    return train_data, test_data

In [5]:
data_augument = keras.Sequential([
                                  preprocessing.RandomFlip('horizontal'),
                                  preprocessing.RandomRotation(0.2),
                                  preprocessing.RandomHeight(0.2),
                                  preprocessing.RandomWidth(0.2),
                                  preprocessing.RandomZoom(0.2),
                                  #preprocessing.Rescaling(1./255) # cho Resnet50V2, không cần thiết với EfficientNetB0
], name="data_augument")

In [6]:
# setup input_shape and base_model
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

16711680/16705208 [==============================] - 0s 0us/step


In [7]:
def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
    # Fine tune model
    base_model.trainable = False
    # tạo layers
    inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
    # thêm data_augument như một layer
    x = data_augument(inputs)
    # give model inputs (after augument) nhưng không train
    x = base_model(x, training= False)
    # Pool layer
    x = layers.GlobalAveragePooling2D(name='pooling_layers')(x)
    # put a dense layer as a out_put
    outputs = layers.Dense(num_classes, activation='softmax', name='output_layers')(x)

    # model từ input và output
    model = keras.Model(inputs, outputs)

    model.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    model.summary()

    return model

In [22]:
# Hàm import ảnh và resize ảnh
def load_and_prep_img(filename, img_shape=224, scale=False):
    # đọc ảnh
    img = tf.io.read_file(filename)
    # dịch ảnh sang tensor unit8
    img = tf.image.decode_jpeg(img)
    # thay đổi kích thước ảnh
    img = tf.image.resize(img, [img_shape, img_shape])
    # đổi giá trị ảnh về giữa đoạn 0 đến 1
    if scale:
        return img/255
    else:
        return img

Download Data

In [9]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_data.zip")

--2021-03-20 14:17:38--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.208, 172.217.164.176, 172.217.15.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M   244MB/s    in 2.0s    

2021-03-20 14:17:41 (244 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In [10]:
# có bao nhiêu ảnh trong mỗi folder ?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
    print(f"Có {len(dirnames)} thư mục con và có {len(filenames)} ảnh trong thư mục '{dirpath}'.")

Có 2 thư mục con và có 0 ảnh trong thư mục '10_food_classes_all_data'.
Có 10 thư mục con và có 0 ảnh trong thư mục '10_food_classes_all_data/test'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/hamburger'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/sushi'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/chicken_wings'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/fried_rice'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/pizza'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/ice_cream'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/steak'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/ramen'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_data/test/chicken_curry'.
Có 0 thư mục con và có 250 ảnh trong thư mục '10_food_classes_all_d

In [11]:
# hàm tạo nhằm mục đích load lại mỗi khi chạy mới mô hình
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
                log_dir=log_dir
    )
    print(f"Lưu lại thông tin ở: {log_dir}")
    return tensorboard_callback

    Vì ta có khả năng có thể chạy lại model nhiều lần, vì thế ta có thể theo dõi chúng theo một cách nào đó.
    Hàm trên lưu lại nhật kí hiệu suất mô hình ở thư mục: [dir_name]/[experiment_name]/[current_timestamp]:
    - dir_name:  thư mục lưu lại nhật kí tổng thể.
    - experiment_name: thử nghiệm cụ thể.
    - current_timestamp: thời gian thử nghiệm chi tiết.

Model 1 (10 classes)

In [12]:
train_data, test_data = create_data_loader(train_dir="10_food_classes_all_data/train/",
                                           test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [13]:
# xem kích cỡ dataset
print(train_data)
len(train_data)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>


235

In [14]:
print(test_data)
len(test_data)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>


79

In [15]:
# call model
model_1 = create_model(num_classes=len(train_data.class_names))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
data_augument (Sequential)   (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layers (GlobalAverag (None, 1280)              0         
_________________________________________________________________
output_layers (Dense)        (None, 10)                12810     
Total params: 4,062,381
Trainable params: 12,810
Non-trainable params: 4,049,571
_________________________________________________________________


In [16]:
history_1_percent = model_1.fit(train_data,
                    epochs=10,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)),
                    callbacks = [create_tensorboard_callback('transfer_learning', 'all_data_aug')]
                    )

Lưu lại thông tin ở: transfer_learning/all_data_aug/20210320-141747
Epoch 1/10
235/235 [==============================] - 120s 354ms/step - loss: 1.4108 - accuracy: 0.5702 - val_loss: 0.5697 - val_accuracy: 0.8289
Epoch 2/10
235/235 [==============================] - 62s 262ms/step - loss: 0.6990 - accuracy: 0.7828 - val_loss: 0.4821 - val_accuracy: 0.8421
Epoch 3/10
235/235 [==============================] - 60s 252ms/step - loss: 0.6137 - accuracy: 0.8061 - val_loss: 0.4530 - val_accuracy: 0.8602
Epoch 4/10
235/235 [==============================] - 54s 230ms/step - loss: 0.5899 - accuracy: 0.8107 - val_loss: 0.4280 - val_accuracy: 0.8586
Epoch 5/10
235/235 [==============================] - 51s 214ms/step - loss: 0.5354 - accuracy: 0.8283 - val_loss: 0.4182 - val_accuracy: 0.8684
Epoch 6/10
235/235 [==============================] - 50s 213ms/step - loss: 0.5026 - accuracy: 0.8453 - val_loss: 0.4534 - val_accuracy: 0.8569
Epoch 7/10
235/235 [==============================] - 48s 202

In [17]:
# số lớp(labels) có thể dự đoán được.
class_name = train_data.class_names
class_name

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [18]:
# test model
# load ảnh
!wget https://img-global.cpcdn.com/recipes/249855c4b55f69ca/751x532cq70/hamburger-heo-bo-nha-lam-recipe-main-photo.jpg

--2021-03-20 14:37:04--  https://img-global.cpcdn.com/recipes/249855c4b55f69ca/751x532cq70/hamburger-heo-bo-nha-lam-recipe-main-photo.jpg
Resolving img-global.cpcdn.com (img-global.cpcdn.com)... 151.101.2.132, 151.101.66.132, 151.101.130.132, ...
Connecting to img-global.cpcdn.com (img-global.cpcdn.com)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34551 (34K) [image/jpeg]
Saving to: ‘hamburger-heo-bo-nha-lam-recipe-main-photo.jpg’

hamburger-heo-bo-nh 100%[===================>]  33.74K  --.-KB/s    in 0.001s  

2021-03-20 14:37:04 (57.5 MB/s) - ‘hamburger-heo-bo-nha-lam-recipe-main-photo.jpg’ saved [34551/34551]



In [23]:
# Xử lí lại ảnh
hamburger_img = load_and_prep_img("hamburger-heo-bo-nha-lam-recipe-main-photo.jpg")

In [24]:
hamburger_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[162.55135 , 166.55135 , 167.55135 ],
        [165.54031 , 169.54031 , 170.54031 ],
        [163.76688 , 167.76688 , 168.76688 ],
        ...,
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ]],

       [[167.9375  , 171.9375  , 174.9375  ],
        [166.96693 , 170.96693 , 173.96693 ],
        [166.9926  , 170.9926  , 173.9926  ],
        ...,
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ]],

       [[167.07715 , 170.63965 , 174.51465 ],
        [168.10645 , 171.66895 , 175.54395 ],
        [168.86021 , 172.42271 , 176.29771 ],
        ...,
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ]],

       ...,

       [[101.23535 ,  96.23535 , 102.23535 ],
        [100.

In [25]:
# mở rổng ảnh
hamburger_expanded = tf.expand_dims(hamburger_img, axis=0) # mở rộng kích thước ảnh từ (224, 224, 3) --> (1, 224, 224, 3)
predict = model_1.predict(hamburger_expanded)
predict

array([[2.0615580e-06, 1.1825097e-07, 8.6500813e-06, 7.2703160e-06,
        9.9358338e-01, 6.3661477e-03, 2.0537625e-07, 4.0481515e-07,
        7.2573306e-07, 3.0969870e-05]], dtype=float32)

In [26]:
# kiểm tra nó thuộc lớp nào
class_name[tf.argmax(predict[0])]

'hamburger'